# **Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia**

Web scrap Falcon 9 launch records with BeautifulSoup:



*   Extract a Falcon 9 launch records HTML table from Wikipedia
 
*   Parse the table 
and convert it into a Pandas data frame




In [167]:
import sys
from bs4 import BeautifulSoup
import requests
import unicodedata
import csv
# import openpyx1
import pandas as pd
import numpy as np

a few helper functions to help us obtain data through scraping

In [168]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass
    
def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name   

In [169]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

## Creating a request for the Falcon9 Launch Wiki page from its URL




In [170]:
response = requests.get(static_url).text

Creating a BeautifulSoup object from the HTML response

In [171]:
soup = BeautifulSoup(response, 'html.parser')

Printing the page title to verify if the BeautifulSoup object was created properly

In [172]:
print(soup.title)

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>


 Extracting all column/variable names from the HTML table header

In [173]:
html_tables = soup.find_all("table")

Extract column name one by one

In [174]:
column_names = []
temp = soup.find_all('th')
for x in range(len(temp)):
  try:
    name = extract_column_from_header(temp[x])
    if (name is not None and len(name) > 0):
      column_names.append(name)
  except:
   pass

Creating a data frame by parsing the launch 

In [175]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
# del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Date']=[]
launch_dict['Time']=[]

fill up the launch_dict with launch records extracted from table rows.

In [177]:
extracted_row = 0
#Extract each table 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        row=rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
              # Flight Number value
      
            launch_dict["Flight No."].append(flight_number)

            datatimelist=date_time(row[0])
            # Date value
        
            date = datatimelist[0].strip(',')
            launch_dict["Date"].append(date)
    
            
            # Time value
        
            time = datatimelist[1]
            launch_dict["Time"].append(time)
             # Booster version

            # bv=booster_version(row[1])
            # if not(bv):
            #     bv=row[1].a.string
            # launch_dict["Version Booster"].append(bv)
 
            
            # Launch Site
       
            launch_site = row[2].a.string
            launch_dict["Launch site"].append(launch_site)
            # Payload
  
            payload = row[3].a.string
            launch_dict["Payload"].append(payload)

            
            # Payload Mass
         
            payload_mass = get_mass(row[4])
            launch_dict["Payload mass"].append(payload_mass)
  
            
            # Orbit

            orbit = row[5].a.string
            launch_dict["Orbit"].append(orbit)
            
            # Customer
      
            customer = row[6].a.string
            launch_dict["Customer"].append(customer)

            # Launch outcome
       
            launch_outcome = list(row[7].strings)[0]
            launch_dict["Launch outcome"].append(launch_outcome)


AttributeError: ignored

create a dataframe

In [178]:
headings = []
for key,values in dict(launch_dict).items():
    if key not in headings:
        headings.append(key)
    if values is None:
        del launch_dict[key]

def pad_dict_list(dict_list, padel):
    lmax = 0
    for lname in dict_list.keys():
        lmax = max(lmax, len(dict_list[lname]))
    for lname in dict_list.keys():
        ll = len(dict_list[lname])
        if  ll < lmax:
            dict_list[lname] += [padel] * (lmax - ll)
    return dict_list

pad_dict_list(launch_dict,0)

df = pd.DataFrame.from_dict(launch_dict)
df.head()


,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Date,Time
0,1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success\n,4 June 2010,18:45
1,2,CCAFS,Dragon,0,LEO,NASA,Success,8 December 2010,15:43
2,3,CCAFS,Dragon,525 kg,LEO,NASA,Success,22 May 2012,07:44
3,4,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success\n,8 October 2012,00:35
4,5,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success\n,1 March 2013,15:10


In [179]:
df.isnull().sum()

Flight No.        0
Launch site       0
Payload           0
Payload mass      0
Orbit             0
Customer          2
Launch outcome    0
Date              0
Time              0
dtype: int64

In [180]:
df['Customer']=df['Customer'].replace(np.nan, "Failure")

In [181]:
df.isnull().sum()

Flight No.        0
Launch site       0
Payload           0
Payload mass      0
Orbit             0
Customer          0
Launch outcome    0
Date              0
Time              0
dtype: int64

In [182]:
df.to_csv('spacex_web_scraped.csv', index=False)